In [ ]:
#import polars as pl
#from datasets import Dataset, load_dataset
#import torch

In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
mt5_tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
mt5_model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
from transformers import MBartForConditionalGeneration, AutoTokenizer
mbart_tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-cc25")
mbart_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
gpt2_tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
gpt2_model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [ ]:
def answer_question(question: str, context: str, model, tokenizer) -> str:
    promt = f"question: {question}  context: {context}"
    input_tokens = tokenizer(promt, return_tensors="pt", truncation=True, max_length=512)
    generated_tokens = model.generate(
        **input_tokens,
        max_new_tokens=64,
        #num_beams=4, # Beam search see lab 4
        early_stopping=True,
        top_p=0.95, temperature=0.9 # 
    )
    answer = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    return answer

In [ ]:
answer_question("What is the best way to fall asleep?", "do not drink a white monster right before going to bed", t5_model, t5_tokenizer)

In [ ]:
answer_question("What is the best way to fall asleep?", "do not drink a white monster right before going to bed", mt5_model, mt5_tokenizer)

In [ ]:
answer_question("What is the best way to fall asleep?", "do not drink a white monster right before going to bed", mbart_model, mbart_tokenizer)

In [ ]:
answer_question("What is the best way to fall asleep?", "do not drink a white monster right before going to bed", gpt2_model, gpt2_tokenizer)